# Initialize the engine and map

In [ ]:
import ee
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

import geemap

geemap package not installed. Installing ...


In [ ]:
!pip 

In [ ]:
import geemap
center=[29.8692992,-95.5408542]
Map = geemap.Map(center=center, zoom=11)
Map.add_basemap('SATELLITE') # Add Google Map
Map

Map(center=[29.8692992, -95.5408542], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [ ]:
import ee
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

# define search parameters

In [ ]:
date = ee.Date("2000-09-01")
print(date.get('Year').getInfo())

2000


In [ ]:
#search parameters are set here
from datetime import date, timedelta #these are standard python datatime classes
pnt_date = date(2019, 10, 23)
delta = timedelta(days = 10)

roi = ee.Geometry.Polygon(
        [[[-95.672374, 29.973996],
          [-95.672374, 29.754586],
          [-95.381226, 29.754586],
          [-95.381226, 29.973996]]])

center = roi.centroid()
lon,lat = center.coordinates().getInfo()
bands = ['B1','B2','B3','B4','B5','B7']
vis = {'bands': ['B4', 'B3', 'B2'],'min': [0, 0, 0], 'max': [20000,20000, 20000]} #reflectance * 10000

In [ ]:
geojsonObject = { "type": "Polygon", "coordinates":  [[[-95.672374, 29.973996],
          [-95.672374, 29.754586],
          [-95.381226, 29.754586],
          [-95.381226, 29.973996]]]}
p1 = ee.Geometry(geojsonObject)
print(p1.area().getInfo())


684958139.491315


In [ ]:
Map.addLayer(p1,{},"o1")

NameError: ignored

# Search for the image

In [ ]:
pnt_date = date(2013, 5, 30)
LC8_header = 'LANDSAT/LC08/C02/T1_L2'
l8_bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
if delta.days == 0:
    start = pnt_date
    end = pnt_date + datetime.timedelta(days = 1)
else:
    start = pnt_date - delta
    end = pnt_date + delta
date1 = ee.Date.fromYMD(start.year, start.month, start.day)
date2 = ee.Date.fromYMD(end.year, end.month, end.day)    
poi = ee.Geometry.Point(lon, lat)
images = ee.ImageCollection(LC8_header) \
    .filterBounds(poi) \
    .filterDate(date1, date2) \
    .sort('CLOUD_COVER')

try:
    images.getInfo()
    image1 = images.first()
except:
    print('No image was found, try another date')
ac_date = ee.Date(image1.get('system:time_start')).format('MM/dd/YYYY')
cloud_cover = image1.get('CLOUD_COVER').getInfo()
print('Found an ' + ' image for date:', pnt_date, ' at date:', ac_date.getInfo(), 'cloud cover = ', cloud_cover)
img1 = image1.select(l8_bands,bands)
image_studyarea = img1.clip(roi)
Map.addLayer(image_studyarea, vis,'clipped')

Found an  image for date: 2013-05-30  at date: 06/05/2013 cloud cover =  16.22


In [ ]:
image1.getInfo()

{'bands': [{'crs': 'EPSG:32614',
   'crs_transform': [30, 0, 687285, 0, -30, 3465015],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7281, 7251],
   'id': 'SR_B1'},
  {'crs': 'EPSG:32614',
   'crs_transform': [30, 0, 687285, 0, -30, 3465015],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7281, 7251],
   'id': 'SR_B2'},
  {'crs': 'EPSG:32614',
   'crs_transform': [30, 0, 687285, 0, -30, 3465015],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7281, 7251],
   'id': 'SR_B3'},
  {'crs': 'EPSG:32614',
   'crs_transform': [30, 0, 687285, 0, -30, 3465015],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7281, 7251],
   'id': 'SR_B4'},
  {'crs': 'EPSG:32614',
   'crs_transform': [30, 0, 687285, 0, -30, 3465015],
   'data_type': {'max

In [ ]:
print(image1.select(['SR_B7', 'SR_B5', 'SR_B3'], ['SWIR1', 'NIR', 'Green']).bandNames().getInfo())

['SWIR1', 'NIR', 'Green']


# Apply cloud mask

In [ ]:
mask = image1.select(['QA_PIXEL'])
cloudless = mask.bitwiseAnd(1<<3).eq(0)
image_masked = image1.updateMask(cloudless)

In [ ]:
Map.addLayer(image_masked, {'bands': ['SR_B5', 'SR_B4', 'SR_B2'],'min': [0, 0, 0], 'max': [20000,20000, 20000]} ,'image_masked')

# export the image

In [ ]:
#export image for download
image_descrition = "ls8_2013-04-01"
image_name = "ls8_2013-04-01"
folder_name = "Landsat8"
file_extension = "GeoTIFF"
#download the image chip to google drive
task1 = ee.batch.Export.image.toDrive(image=image_studyarea,description=image_descrition,fileNamePrefix=image_name
                                      ,fileFormat=file_extension,scale=30, folder=folder_name)
task1.start()

# compute the NDVI

In [ ]:
def getNDVI(image):
    return image.normalizedDifference(['B4', 'B3']) #for landsat 8, band 5 is nir and band 4 is red
def getNDVI1(image):
  return image.expression('(b("B4") - b("B3")) / (b("B4") + b("B3"))')  
ndvi1 = getNDVI1(image_studyarea)
vizParams = {'min': -0.5, 'max': 0.5,
             'palette': ['blue', 'yellow', 'green']}
Map.addLayer(ndvi1, vizParams, 'NDVI')